In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive/') 
os.chdir('/content/gdrive/My Drive/Colab Notebooks/챗봇')

Mounted at /content/gdrive/


In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 282kB/s 
     |████████████████████████████████| 460kB 37.5MB/s 
     |████████████████████████████████| 92kB 9.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
import tensorflow as tf
import numpy as np

from konlpy.tag import Okt
import pandas as pd
import tensorflow as tf
import enum
import os
import re
import json
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

from preprocess import *

In [4]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [7]:
PATH = 'data_in/ChatBotData.csv'
VOCAB_PATH = 'data_in/vocabulary.txt'

inputs, outputs = load_data(PATH)
char2idx, idx2char, vocab_size = load_vocabulary(PATH, VOCAB_PATH, tokenize_as_morph=True, method='Okt')

  0%|          | 0/11823 [00:00<?, ?it/s]

Okt


100%|██████████| 11823/11823 [00:13<00:00, 859.48it/s]


In [8]:
index_inputs, input_seq_len = enc_processing(inputs, char2idx, tokenize_as_morph=True, method='Okt')
index_outputs, output_seq_len = dec_output_processing(outputs, char2idx, tokenize_as_morph=True, method='Okt')
index_targets = dec_target_processing(outputs, char2idx, tokenize_as_morph=True, method='Okt')

  2%|▏         | 210/11823 [00:00<00:05, 2099.51it/s]

Okt


  1%|▏         | 156/11823 [00:00<00:07, 1545.17it/s]

Okt


  0%|          | 0/11823 [00:00<?, ?it/s]

Okt


100%|██████████| 11823/11823 [00:13<00:00, 868.55it/s]


> ** Okt 문장그대로 **

In [9]:
vocab_size

12657

In [10]:
data_configs = {}
data_configs['char2idx'] = char2idx
data_configs['idx2char'] = idx2char
data_configs['vocab_size'] = vocab_size
data_configs['pad_symbol'] = PAD
data_configs['std_symbol'] = STD
data_configs['end_symbol'] = END
data_configs['unk_symbol'] = UNK

DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json'

np.save(open(DATA_IN_PATH + TRAIN_INPUTS, 'wb'), index_inputs)
np.save(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'wb'), index_outputs)
np.save(open(DATA_IN_PATH + TRAIN_TARGETS , 'wb'), index_targets)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'))

In [11]:
index_inputs = np.load(open(DATA_IN_PATH + TRAIN_INPUTS, 'rb'))
index_outputs = np.load(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'rb'))
index_targets = np.load(open(DATA_IN_PATH + TRAIN_TARGETS , 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

In [12]:
char2idx = prepro_configs['char2idx']
end_index = prepro_configs['end_symbol']
model_name = 'transformer'
vocab_size = prepro_configs['vocab_size']
BATCH_SIZE = 2
MAX_SEQUENCE = 25
EPOCHS = 30
VALID_SPLIT = 0.1

kargs = {
         'num_layers': 2,
         'd_model': 512,
         'num_heads': 8,
         'dff': 2048,
         'input_vocab_size': vocab_size,
         'target_vocab_size': vocab_size,
         'maximum_position_encoding': MAX_SEQUENCE,
         'end_token_idx': char2idx[end_index],
         'rate': 0.1
        }

# 패딩 및 포워드 마스킹

In [13]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

# 순방향 마스크 어텐션

In [ ]:
def create_look_ahead_mask(size):
  mask = 1- tf.linalg.band_part(input=tf.ones((size,size)), num_lower=-1, num_upper=0)
  return mask

In [ ]:
def scaled_dot_product_attention(q,k,v,mask):
  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (batch, seq_len_q, seq_len_k)
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(df)

  if mask is not None:
    scaled_attention_logits += (mask * -1e9)
  
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1) # (batch, seq_len_q, seq_len_k)
  output = tf.matmul(attention_weights, v)  # # (batch, seq_len_q, depth_v)
  return output, attention_weights

# 멀티헤드 어텐션

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, **kargs):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = kargs['num_heads']
    self.d_model = kargs['d_model']

    assert self.d_model % self.num_heads == 0

    self.depth = self.d_model // self.num_heads

    # 스케일 내적 연산 이전에 입력한 key, query, value에 대한 차원수를 맞추기 위한 레이어
    self.wq = tf.keras.layers.Dense(kargs['d_model'])
    self.wk = tf.keras.layers.Dense(kargs['d_model'])
    self.wv = tf.keras.layers.Dense(kargs['d_model'])

    # 셀프 어텐션 레이어를 출력하기 위한 레이어다. 
    self.dense = tf.keras.layers.Dense(kargs['d_model'])

  # 이번에는 각 벡터를 헤드 수만큼 나눌 수 있게 한다.
  # key, query, value에 대한 벡터를 헤드 수만큼 분리할 수 있게 하는 함수
  # [batch, sequence, feature] -> [batch, head, sequence, feature]
  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0,2,1,3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  #(batch, sequnce, d_model)
    k = self.wk(k)  #(batch, sequnce, d_model)
    v = self.wv(v)  #(batch, sequnce, d_model)

    q = self.split_heads(q, batch_size)  # (batch, num_heads, seq, depth)
    k = self.split_heads(k, batch_size)
    v = self.split_heads(v, batch_size)

    #scaled_attention.shape == (batch, num, seq_len_q, depth)    
    #attention_weights.shape == (batch, num, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(q,k,v,mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0,2,1,3])
    #(batch, seq_len_q, num, depth)
    concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
    # (batch, seq_len_q, d_model)
    output = self.dense(concat_attention) # (batch, seq_len_q, d_model)

    return output, attention_weights




  



# 포인트 와이즈 피드포워드 네트워크

$ FFN(x) = max(0, xW_1 + b_1)W_2 + b_2 $

- 한 문장 안에 있는 단어 토큰 벡터 각각에 대해 연산하는 네트워크
- 두 개의 linear 레이어를 거치고 그 사이 활성화 함수로 ReLU를 활용.

In [ ]:
def point_wise_feed_forward_network(**kargs):
  return tf.keras.Sequential([
                              tf.keras.layers.Dense(kargs['dff'], activation='relu'),
                              tf.keras.layers.Dense(kargs['d_model'])])

# Residual Connection

```python
self.ffn = point_wise_feed_forward_network(d_model, dff)
self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

ffn_output = self.ffn(out1)
out2 = self.layernorm2(out1 + ffn_output)
```

- 인코더 레이어에서 리지듀얼 커넥션을 수행하는 부분을 보여준다.
- 먼저 연산할 레이어 피드 포워드 네트워크와 레이어 노멀라이즈를 각각 생성한다.
- 레이어 노멀라이즈는 각 레이어별 뉴런을 노멀라이즈하는 역할이라고 이해하면 된다.
- 피드 포워드 네트워크를 만들기 위해서는 앞서 구현한 point_wise_feed_forward_network를 통해 네트워크를 생성한다.

> - 입력 벡터 out1을 먼저 self.ffn을 통해 피드 포워드 연산을 하게 해서 ffn_outpu으로 할당한다. 그 다음 입력 벡터 out1과 피드 포워드 네트워크를 거쳐 나온 ffn_output을 더해 리지듀얼 커넥션을 수행한다.


> - 이게 transformer모델에서 **Add&Norm**에 해당

# 포지션 인코딩

- 트랜스포머는 seq2seq와 달리 단어 하나하나 순차적으로 넣지 않고 한번에 넣음. 
- 따라서 입력 시퀀스 정보에 대한 순서 정보를 부가적으로 주입해야 함
- 여러 방법이 있으나 논문방법은

$$ PE_{(pos,2i)} = sin(pos/10000^{{2i}/d_{model}}) $$

$$ PE_{(pos,2i+1)} = cos(pos/10000^{{2i}/d_{model}}) $$

- 첫번째 수식은 피처 차원에서 인덱스가 짝수인 경우에 대해 사인 함수값 할당
- 두번째는 피처 차원에서 인덱스가 홀수인 경우에 코사인 값 할당
- 함수 안에 있는 식은 각 시퀀스 위치에 따라 피처 차원 인덱스에 각자 위치 정보를 달리 주고자 하는 의도


In [ ]:
# pos/10000^(2i/dim) 에 해당
# pos는 포지션에 대한 인덱스 위치 리스트
# i에는 차원에 대한 리스트
# 각 순서에 따른 각도값을 얻는다.

def get_angles(pos, i, d_model):
  angle_rates= 1/np.power(10000, (2*i) / np.float32(d_model))
  return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  angle_rads[:, 0::2] = np.sin(angle_rads[:,0::2])
  angle_rads[:, 1::2] = np.cos(angle_rads[:,1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

# Encoder Layer

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, **kargs):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(**kargs)
    self.ffn = point_wise_feed_forward_network(**kargs)

    self.layersnorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layersnorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(kargs['rate'])
    self.dropout2 = tf.keras.layers.Dropout(kargs['rate'])

  def call(self, x, mask):
    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output)
    out1 = self.layernorm1(x + attn_output)

    ffn_output = self.ffn(out1) # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output) #(batch, input_seq_len, d_model)

    return out2

# Encoder

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, **kargs):
    super(Encoder, self).__init__()

    self.d_model = kargs['d_model']
    self.num_layers = kargs['num_layers']

    self.embedding = tf.keras.layers.Embedding(kargs['input_vocab_size'], self.d_model)    
    self.pos_encdoing = positional_encoding(kargs['maximum_position_encoding'], self.d_model)

    self.enc_layers = [EncoderLayer(**kargs) for _ in range(self.num_layers)]

    self.dropout = tf.keras.layers.Dropout(kargs['rate'])

  def call(self, x, mask):

    seq_len = tf.shape(x)[1]

    x = self.embedding(x)

    # 임베딩이 할당된 후에 임베딩 차원수의 제곱근만큼에 대한 가중치 곱함
    # 각 워드 임베딩에 대해 스케일을 맞추기 위함
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, mask)

    return x #batch, input_seq_len, d_model

# Decoder 레이어

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(**kargs)
        self.mha2 = MultiHeadAttention(**kargs)

        self.ffn = point_wise_feed_forward_network(**kargs)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(kargs['rate'])
        self.dropout2 = tf.keras.layers.Dropout(kargs['rate'])
        self.dropout3 = tf.keras.layers.Dropout(kargs['rate'])
    
    
    def call(self, x, enc_output, look_ahead_mask, padding_mask):
        # enc_output.shape == (batch_size, input_seq_len, d_model)
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3, attn_weights_block1, attn_weights_block2

# Decoder

In [ ]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, **kargs):
        super(Decoder, self).__init__()

        self.d_model = kargs['d_model']
        self.num_layers = kargs['num_layers']

        self.embedding = tf.keras.layers.Embedding(kargs['target_vocab_size'], self.d_model)
        self.pos_encoding = positional_encoding(kargs['maximum_position_encoding'], self.d_model)

        self.dec_layers = [DecoderLayer(**kargs) 
                           for _ in range(self.num_layers)]
        self.dropout = tf.keras.layers.Dropout(kargs['rate'])

    def call(self, x, enc_output, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        # x.shape == (batch_size, target_seq_len, d_model)
        return x, attention_weights

# Transformer

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, **kargs):
        super(Transformer, self).__init__()
        self.end_token_idx = kargs['end_token_idx']
        
        self.encoder = Encoder(**kargs)
        self.decoder = Decoder(**kargs)

        self.final_layer = tf.keras.layers.Dense(kargs['target_vocab_size'])

    def call(self, x):
        inp, tar = x

        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
        enc_output = self.encoder(inp, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

        # dec_output.shape == (batch_size, tar_seq_len, d_model)
        dec_output, _ = self.decoder(
            tar, enc_output, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

        return final_output
    
    def inference(self, x):
        inp = x
        tar = tf.expand_dims([STD_INDEX], 0)

        enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)        
        enc_output = self.encoder(inp, enc_padding_mask)
        
        predict_tokens = list()
        for t in range(0, MAX_SEQUENCE):
            dec_output, _ = self.decoder(tar, enc_output, look_ahead_mask, dec_padding_mask)
            final_output = self.final_layer(dec_output)
            outputs = tf.argmax(final_output, -1).numpy()
            pred_token = outputs[0][-1]
            if pred_token == self.end_token_idx:
                break
            predict_tokens.append(pred_token)
            tar = tf.expand_dims([STD_INDEX] + predict_tokens, 0)
            _, look_ahead_mask, dec_padding_mask = create_masks(inp, tar)
            
        return predict_tokens

# 모델 로스 정의

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [ ]:
model = Transformer(**kargs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=loss,
              metrics=[accuracy])

In [ ]:
# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=10)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)

checkpoint_path = DATA_OUT_PATH + model_name + '/weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    

cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

./data_out/transformer -- Folder already exists 

